In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)


# Build Classic Transform Using Least Squares

Use standard field to calculate photometric transform by classic method ("slope of slopes")

In [ ]:
import vso.util
import vso.data
import vso.phot
import numpy as np

OBJECTS = ['SA20', 'SA38', 'SA41']
SESSION_TAG='2024/20241006'
IMAGE_ROOT = '/srv/public/img'
WORK_ROOT = '/srv/public'

layout = vso.util.WorkLayout(WORK_ROOT)
sd = vso.data.StarData(layout.charts_dir)


In [ ]:
from astropy.table import QTable, Column, vstack, join

bands = ('B', 'V')
def get_session_data(name, tag, bands):
    session = vso.util.Session(tag=tag, name=name)
    layout = vso.util.WorkLayout(WORK_ROOT)
    session_layout = layout.get_session(session)
    provider = vso.phot.BatchDataProvider(session_layout)
    return provider.batch_and_sequence_band_pair(bands)


In [ ]:
from vso.phot import ValErr, ValErrDtype, MagErrDtype
from scipy import stats as sst
def get_star_data(name, tag, bands):

    # A = [C1-C2, -X*(C1-C2)] = [dC, -X*dC];
    #  x = [T, k2];
    #  B = [(M1-M2) - (m1-m2)] = [dM - dm]

    def build_diff_sequence(sequence, selector, bands):
        A11 = QTable({
            'auid_1': sequence['auid'],
            f'{bands[0]}_1': sequence[bands[0]],
            'C_1': sequence['C']
        })
        A12 = QTable({
            'auid_2': sequence['auid'],
            f'{bands[0]}_2': sequence[bands[0]],
            'C_2': sequence['C']
        })
        A21 = QTable({
            'auid_1': sequence['auid'],
            f'{bands[1]}_1': sequence[bands[1]]
        })
        A22 = QTable({
            'auid_2': sequence['auid'],
            f'{bands[1]}_2': sequence[bands[1]]
        })
        A0 = join(join(join(join(selector, A11, 'auid_1'), A12, 'auid_2'), A21, 'auid_1'), A22, 'auid_2')
        A0[f'd{bands[0]}'] = A0[f'{bands[0]}_1']['mag'] - A0[f'{bands[0]}_2']['mag']
        A0[f'd{bands[1]}'] = A0[f'{bands[1]}_1']['mag'] - A0[f'{bands[1]}_2']['mag']
        A0['dC'] = A0['C_1']- A0['C_2']
        return A0['auid_1', 'auid_2', f'd{bands[0]}', f'd{bands[1]}', 'dC']

    def build_diff_measurement(measured, selector, bands):
        m11 = QTable({
            'batch_id': measured['batch_id'],
            'auid_1': measured['auid'],
            f'{bands[0]}_1': measured[f'instr {bands[0]}']
        })
        m12 = QTable({
            'batch_id': measured['batch_id'],
            'auid_2': measured['auid'],
            f'{bands[0]}_2': measured[f'instr {bands[0]}']
        })
        m21 = QTable({
            'batch_id': measured['batch_id'],
            'auid_1': measured['auid'],
            f'{bands[1]}_1': measured[f'instr {bands[1]}']
        })
        m22 = QTable({
            'batch_id': measured['batch_id'],
            'auid_2': measured['auid'],
            f'{bands[1]}_2': measured[f'instr {bands[1]}']
        })

        m = join(join(join(join(selector, m11, ['batch_id', 'auid_1']), m12, [
                 'batch_id', 'auid_2']), m21, ['batch_id', 'auid_1']), m22, ['batch_id', 'auid_2'])
        m[f'instr d{bands[0]}'] = m[f'{bands[0]}_1']['mag'] - m[f'{bands[0]}_2']['mag']
        m[f'instr d{bands[1]}'] = m[f'{bands[1]}_1']['mag'] - m[f'{bands[1]}_2']['mag']
        return m['batch_id', 'auid_1', 'auid_2', f'instr d{bands[0]}', f'instr d{bands[1]}']

    session = vso.util.Session(tag=tag, name=name)
    layout = vso.util.WorkLayout(WORK_ROOT)
    session_layout = layout.get_session(session)
    provider = vso.phot.BatchDataProvider(session_layout)

    sequence = provider.sequence_band_pair(bands)
    sequence['C'] = Column(sequence[bands[0]]['mag'] - sequence[bands[1]]['mag'])

    stars = QTable(dict(auid = sequence['auid']))
    cartesian = join(stars, stars, join_type='cartesian')
    sequence_selector = cartesian[cartesian['auid_1'] != cartesian['auid_2']]

    batches = QTable(dict(batch_id = provider.batches_['batch_id']))
    batch_sequence_selector = join(batches, sequence_selector, join_type='cartesian')
    A0 = build_diff_sequence(sequence, sequence_selector, bands)
    measured = provider.batch_band_pair(bands)
    diff_measured = build_diff_measurement(measured, batch_sequence_selector, bands)
    result = join(join(diff_measured, A0, ['auid_1', 'auid_2']), provider.batches_['batch_id', 'airmass'], 'batch_id')
    return result

get_star_data(OBJECTS[0], SESSION_TAG, bands)

In [ ]:
import numpy as np
from scipy import linalg as sla
from collections import namedtuple

ClassicDiffTransform = namedtuple('ClassicDiffTransform', ['band', 'Ta', 'ka', 'Tb', 'kb'])

def calc_classic_diff_transform(data, bands):
    A = np.array([data['dC'], -data['airmass']*data['dC']]).T
    Qjj = np.diag(np.linalg.inv( np.matmul(A.T, A)))
    B1 = data[f'd{bands[0]}'] - data[f'instr d{bands[0]}']
    B2 = data[f'd{bands[1]}'] - data[f'instr d{bands[1]}']
    X1, res1, _, _ = sla.lstsq(A, B1) # [Ta, ka], ...
    X2, res2, _, _ = sla.lstsq(A, B2) # [Tb, kb], ...
    # see https://en.wikipedia.org/wiki/Ordinary_least_squares#Intervals
    err1 = np.sqrt( Qjj*res1/(len(data)-2)) # [Ta_err, ka_err]
    err2 = np.sqrt( Qjj*res2/(len(data)-2)) # [Tb_err, kb_err]
    return ClassicDiffTransform(bands,
                                ValErr(X1[0], err1[0]), ValErr(X1[1], err1[1]),
                                ValErr(X2[0], err2[0]), ValErr(X2[1], err2[1]))


data = vstack([get_star_data(obj, SESSION_TAG, bands) for obj in OBJECTS])

calc_classic_diff_transform(data, bands)